<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Postprocessing - Resampling 1m Vectorisation etc..</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [34]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [35]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import subprocess
import glob

In [36]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [37]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [38]:
run ../SRC/config_postprocess.py

In [39]:
print config_parameters

{'list_tiles': '../../../Postprocess_V1/list_tiles', 'outputfolder_Logfile': '../../../Postprocess_V1/Log_file', 'permanent_mapset': 'PERMANENT', 'locationepsg': '31370', 'outputfolder': '../../../Postprocess_V1', 'outputfolder_Vecteur': '../../../Postprocess_V1/Vecteur', 'gisdb': '../../GRASSDATA', 'location': 'WALOUS_31370', 'PYTHONLIB': '/usr/bin/python2', 'njobs': 6, 'outputfolder_Raster': '../../../Postprocess_V1/Raster', 'GISBASE': '/usr/lib/grass76'}


In [40]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [41]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/grass76/bin:/usr/lib/grass76/script:/usr/lib/grass76/lib:/usr/lib/grass76/bin:/usr/lib/grass76/scripts:/home/tais/.grass7/addons/scripts:/home/tais/.grass7/addons/bin:/usr/lib/grass76/bin:/usr/lib/grass76/script:/usr/lib/grass76/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2075 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/2003,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/2003 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	

** GRASS GIS Python libraries **

In [42]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Import libraries**

In [43]:
# Import function that check and create folder
from mkdir import check_create_dir

In [44]:
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name

## Special functions

In [45]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time

In [46]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [47]:
def GetMapsetsAccess():
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Add mapsets with input data to the GRASS GIS research path
        gscript.run_command('g.mapsets', mapset="FUSIONS", operation="add")
        return_message = "Access to other mapset added"
    except:
        return_message += "ERROR: Add access to other Mapsets failed. Please check for problem."
    return return_message

In [48]:
def DefineComputationRegion(tile_cat,resamp_resolution='1', grow_mmu='16'):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        return_message = "Working on tile '%s'\n"%tile_cat
        # 'Grow' parameter is used to expend the computational region ouside of the current tile to deal with bordering objects smaller than MMU
        gscript.run_command('g.region', flags='a', raster='RF_fusion_tile_%s'%tile_cat, 
                            res=resamp_resolution, grow=grow_mmu)  
        # Print
        return_message += "--> Computational region defined"
    except:
        return_message += "ERROR: Setting of computional region failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [49]:
def Resamp(tile_cat):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('r.resamp.stats', overwrite=True, input='RF_fusion_tile_%s'%tile_cat, 
                            output='resamp', method='mode')
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Resampling for cutline '%s' failed. Please check for problem."%tile_cat
    return return_message   

In [50]:
def ReclassArea(tile_cat, MMU='0.0015'):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('r.reclass.area', flags='c', overwrite=True, input='resamp',
                            output='resamp_mmu', value=MMU, mode='lesser', method='rmarea')
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Reclassing area smaller than MMU failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message   

In [51]:
def ColorizeRaster(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        gscript.run_command('r.colors', map='resamp_mmu', rules=data['color_file'])
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Application of colors table on raster failed for tile %s. Please check for problem."%tile_cat
    return return_message

In [52]:
def Vectorize(tile_cat):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('r.to.vect', flags='s', overwrite=True,
                            input='resamp_mmu', output='vect', type='area', column='class')
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Vectorization of raster map failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message   

In [53]:
def VectorGeneralize(tile_cat, thresh_value="10000000", iter_value="3"):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('v.generalize', overwrite=True, input='vect', type='area', output='vect_generalized', 
                            method='chaiken', threshold=thresh_value, iterations=iter_value)
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Vectorization of raster map failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message   

In [54]:
def ColorizeVector(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        gscript.run_command('v.colors', map='vect_generalized', use='attr',
                            column='class', rules=data['color_file'])
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Application of colors table on vector failed for tile %s. Please check for problem."%tile_cat
    return return_message

In [55]:
def ExportRast(tile_cat):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        export_path = os.path.join(config_parameters['outputfolder_Raster'],"LC_resamp_mmu_tile_%s.tif"%tile_cat)
        # Export the group as a .tif file
        gscript.run_command('r.out.gdal', quiet=True, overwrite=True, input="resamp_mmu", output=export_path,
                            format='GTiff', createopt='COMPRESS=DEFLATE') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
        return_message += "--> Raster exported"
    except:
        return_message += "ERROR: Export of raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [75]:
def ExportAllRast(tile_cat):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        export_path = os.path.join(config_parameters['outputfolder_Raster'],"LC_resamp_tile_%s.tif"%tile_cat)
        # Export the group as a .tif file
        gscript.run_command('r.out.gdal', quiet=True, overwrite=True, input="resamp", output=export_path,
                            format='GTiff', createopt='COMPRESS=DEFLATE') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
        export_path = os.path.join(config_parameters['outputfolder_Raster'],"LC_resamp_mmu_tile_%s.tif"%tile_cat)
        gscript.run_command('r.out.gdal', quiet=True, overwrite=True, input="resamp_mmu", output=export_path,
                            format='GTiff', createopt='COMPRESS=DEFLATE') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
        return_message += "--> Raster exported"
    except:
        return_message += "ERROR: Export of raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [56]:
def ExportVect(tile_cat):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        export_path = os.path.join(config_parameters['outputfolder_Vecteur'],"LC_resamp_mmu_tile_%s.gpkg"%tile_cat)
        # Export the group as a .tif file
        gscript.run_command('v.out.ogr', quiet=True, overwrite=True,  input="vect_generalized",
                            output=export_path, format='GPKG') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
        return_message += "--> Vector exported"
    except:
        return_message += "ERROR: Export of raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [76]:
def ExportAllVect(tile_cat):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        export_path = os.path.join(config_parameters['outputfolder_Vecteur'],"LC_resamp_mmu_vect_tile_%s.gpkg"%tile_cat)
        # Export the group as a .tif file
        gscript.run_command('v.out.ogr', quiet=True, overwrite=True,  input="vect",
                            output=export_path, format='GPKG') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
        export_path = os.path.join(config_parameters['outputfolder_Vecteur'],"LC_resamp_mmu_vect_smooth_tile_%s.gpkg"%tile_cat)
        # Export the group as a .tif file
        gscript.run_command('v.out.ogr', quiet=True, overwrite=True,  input="vect_generalized",
                            output=export_path, format='GPKG') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
        return_message += "--> Vector exported"
    except:
        return_message += "ERROR: Export of raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [57]:
def Clean(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        for layer in list_rast_fusion:
            gscript.run_command('g.remove', flags='f', type="raster", name=layer)
        return_message += "--> Mapset cleaned"
    except:
        return_message += "ERROR: during mapset cleaning. Please check for problem."
    return return_message

In [77]:
def Worker(tile_cat):
    import subprocess
    start_tile = start_processing() 
#    print "Start processing on tile %s"%tile_cat
    #Declare empty list for saving output messages
    output_message = [] 
    
    # Launch mapset
    message = launch_mapset(tile_cat)  
    [output_message.append(a) for a in message]
#    print "\n".join(message)

    # Allow access to other mapset 
    message = GetMapsetsAccess()
    output_message.append(message)
#    print message
    
    # Define computional region and mask
    message = DefineComputationRegion(tile_cat,resamp_resolution='1', grow_mmu='16')
    output_message.append(message)
#    print message    

    # Resample
    message = Resamp(tile_cat)
    output_message.append(message)
#    print message 

    # ReclassArea according to MMU
    message = ReclassArea(tile_cat, MMU='0.0015')
    output_message.append(message)
#    print message 

    # ColorizeRaster
    message = ColorizeRaster(tile_cat)
    output_message.append(message)
#    print message 

    # Vectorize
    message = Vectorize(tile_cat)
    output_message.append(message)
#    print message 

    # VectorGeneralize
    message = VectorGeneralize(tile_cat, thresh_value="10000000", iter_value="3")
    output_message.append(message)
#    print message 

    # ColorizeVector
    message = ColorizeVector(tile_cat)
    output_message.append(message)
#    print message 

    # ExportRast 
    #message = ExportRast(tile_cat)
    #output_message.append(message)
#    print message
    
    # ExportAllRast 
    message = ExportAllRast(tile_cat)
    output_message.append(message)
#    print message

    # ExportVect 
    #message = ExportVect(tile_cat)
    #output_message.append(message)
#    print message

    # ExportAllVect 
    message = ExportAllVect(tile_cat)
    output_message.append(message)
#    print message

    # Clean 
    #message = Clean(tile_cat)
    #output_message.append(message)
#    print message

    #Print processing time
    message = print_processing_time(start_tile, "Prediction for tile '%s' achieved in "%tile_cat)
    output_message.append(message)
#    print message
    
    #Export Log file
    fout = open(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_%s.txt"%tile_cat),"w")
    [fout.writelines('%s\n'%content) for content in output_message]
    fout.close()

## Create new directories

In [78]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])
check_create_dir(config_parameters['outputfolder_Logfile'])
check_create_dir(config_parameters['outputfolder_Raster'])
check_create_dir(config_parameters['outputfolder_Vecteur'])

The folder '../../../Postprocess_V1' already exists
The folder '../../../Postprocess_V1/Log_file' already exists
The folder '../../../Postprocess_V1/Raster' already exists
The folder '../../../Postprocess_V1/Vecteur' already exists


# Import fusions

In [79]:
run ../SRC/config_postprocess.py

In [28]:
# Create mapset 
launch_mapset("FUSIONS")

["Location 'WALOUS_31370' already exist",
 "'FUSIONS' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/FUSIONS'"]

In [87]:
# Create a list of paths to files with a specific name prefix
list_file = glob.glob(os.path.join(data['fusion_folder'],"RF_fusion_tile_*.tif"))
print "There are %s paths in the list"%len(list_file)

There are 375 paths in the list


In [88]:
# Get list of cat that are included in the shapefile with tiles to be processed
import geopandas as gpd
gdf = gpd.read_file(data['tiles'][1])
tiles_aoi = list(gdf.cat)
gdf = None

In [89]:
# Keep only tiles number that are both exists in the folder containing all LC .tif files and included in the AOI to be processed
list_file = [(os.path.splitext(os.path.split(x)[-1])[0],x) for x in list_file if int(os.path.splitext(os.path.split(x)[-1])[0].split("_")[-1]) in tiles_aoi]
# Create list of cat to be processed
list_cat = [x[0].split("_")[-1] for x in list_file]
print "%s TIF files will be imported (contained in the AOI to be processed)"%len(list_file)

88 TIF files will be imported (contained in the AOI to be processed)


# Apply functions on each tile

In [92]:
# Set number of cores to use
ncores = 20

In [95]:
# Launch processes in parallel
start_parallel = start_processing()
p = Pool(ncores)
output = p.map(Worker, list_cat[:])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%ncores)

'Computation (on 20 cores) achieved in 4 minutes and 3.3 seconds'

**Check log file for ERRORS**

In [96]:
# Get list of csv with classification feature of individual tiles
import glob
list_log = glob.glob(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_*.txt"))
print "%s log files in the folder"%len(list_log)

88 log files in the folder


In [98]:
# Declare new counter
count = 0
# Declare new list that will contain list of tile with error
tile_error_list = []
# Loop on list of log file
for logfile in list_log:
    got_error = False
    tile_num = os.path.splitext(os.path.basename(logfile))[0].split("_")[-1]
    fin = open(logfile, 'r')
    for row in fin:
        if row[:5] == "ERROR":  # If at least one line have error message, the whole file will be counted as 1 error
            got_error = True
    if got_error:    
        count += 1
        tile_error_list.append(tile_num)  # Add tile number to the list
# Print
print "%s tile(s) faced an ERROR during the processing.\n"%count

# Update tile list with only tiles that have ERROR in log 
print "\n".join(["Error on tile %s"%(a) for a in tile_error_list])

0 tile(s) faced an ERROR during the processing.




In [99]:
gscript.run_command('g.region', flags='p')

0

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Create VRT with all raster products

In [73]:
# Create mapset 
launch_mapset("POSTPROCESS")

["Location 'WALOUS_31370' already exist",
 "'POSTPROCESS' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/POSTPROCESS'"]

In [74]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["resamp@%s"%cat for cat in list_cat]), 
                    output='resamp_vrt')

0

In [72]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["resamp_mmu@%s"%cat for cat in list_cat]), 
                    output='resamp_mmu_vrt')

0

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Merge and dissolve border polygons

In [ ]:
v.db.addcolumn map=test_overlay@3219 columns="class integer"

In [ ]:
v.db.update map=test_overlay@3219 column=class query_column=a_class where="b_class is null"

In [ ]:
v.db.update map=test_overlay@3219 column=class query_column=a_class where="(a_class is not null) AND (b_class is not null) "

In [ ]:
v.dissolve input=test_overlay@3219 column=class output=test_dissolve